In [9]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    """
    Generate a feasible bus planning schedule that minimizes the number of buses used.
    
    Parameters:
    dienstregeling (pandas.DataFrame): DataFrame containing the schedule data with columns:
        'startlocatie', 'vertrektijd', 'eindlocatie', 'buslijn'
    afstandsmatrix (pandas.DataFrame): DataFrame containing distance data with columns:
        'startlocatie', 'eindlocatie', 'min reistijd in min', 'afstand in meters', 'buslijn'
    
    Returns:
    pandas.DataFrame: DataFrame containing the feasible schedule with columns:
        'startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'activiteit', 'buslijn', 'energieverbruik', 'starttijd datum', 'eindtijd datum', 'omloop nummer'
    """
    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )
    
    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')
    
    # Initialize the bus schedules and state of charge (SOC) for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    
    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(row['vertrektijd'], format='%H:%M')
        
        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)
        
        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=row['min reistijd in min'])
        arrival_time = departure_time + travel_time
        
        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available and has sufficient SOC
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption:
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = len(bus_soc)
        
        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit',
            'buslijn': bus_line,
            'energieverbruik': energy_consumption,
            'starttijd datum': departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
            'omloop nummer': available_bus
        })
        
        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption
    
    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.
    
    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)

print(f"Consolidated plan saved to {output_file}")


Consolidated plan saved to feasible_plan_consolidated.xlsx


In [10]:
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    """
    Generate a feasible bus planning schedule that minimizes the number of buses used.
    
    Parameters:
    dienstregeling (pandas.DataFrame): DataFrame containing the schedule data with columns:
        'startlocatie', 'vertrektijd', 'eindlocatie', 'buslijn'
    afstandsmatrix (pandas.DataFrame): DataFrame containing distance data with columns:
        'startlocatie', 'eindlocatie', 'min reistijd in min', 'afstand in meters', 'buslijn'
    
    Returns:
    pandas.DataFrame: DataFrame containing the feasible schedule with columns:
        'startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'activiteit', 'buslijn', 'energieverbruik', 'starttijd datum', 'eindtijd datum', 'omloop nummer'
    """
    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )
    
    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')
    
    # Initialize the bus schedules and state of charge (SOC) for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    
    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(row['vertrektijd'], format='%H:%M')
        
        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)
        
        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=row['min reistijd in min'])
        arrival_time = departure_time + travel_time
        
        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available and has sufficient SOC
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption:
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = len(bus_soc)
        
        # Record the trip in the schedule
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit',
            'buslijn': bus_line,
            'energieverbruik': energy_consumption,
            'starttijd datum': departure_time.strftime('%Y-%m-%d %H:%M:%S'),
            'eindtijd datum': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
            'omloop nummer': available_bus
        })
        
        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption
    
    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.
    
    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Generate the feasible plan
feasible_plan_df = generate_feasible_plan(dienstregeling, afstandsmatrix)

# Save the plan to an Excel file in a single sheet
save_plan_to_excel(feasible_plan_df, output_file)

print(f"Consolidated plan saved to {output_file}")


Consolidated plan saved to feasible_plan_consolidated.xlsx
